In [1]:
!pip install np_utils
!pip install load_dotenv

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
import os
import pandas as pd
import numpy as np
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array
from keras.models import load_model
import urllib

from pathlib import Path
# import re
import np_utils
from dotenv import load_dotenv
load_dotenv()

# Define the path to the folder containing the images to be classified
folder_path = "/content/drive/MyDrive/AWS/house-price-estimation/test"

# # Define the URLs for the model files in the GitHub repository
# model_lamp_url = "https://github.com/Aoubiza-Imane/House_Price/blob/main/model_lamp.h5"
# model_dresser_url = "https://github.com/Aoubiza-Imane/House_Price/blob/main/model_dresser.h5"
# model_bed_url = "https://github.com/Aoubiza-Imane/House_Price/blob/main/model_bed.h5"
# model_chair_url = "https://github.com/Aoubiza-Imane/House_Price/blob/main/model_chair.h5"

# # Download the model files from the GitHub repository
# urllib.request.urlretrieve(model_lamp_url, "model_lamp.h5")
# urllib.request.urlretrieve(model_dresser_url, "model_dresser.h5")
# urllib.request.urlretrieve(model_bed_url, "model_bed.h5")
# urllib.request.urlretrieve(model_chair_url, "model_chair.h5")

# Load the pre-trained models
model_lamp = tf.keras.models.load_model("/content/drive/MyDrive/AWS/model_lamp.h5")
model_dresser = tf.keras.models.load_model("/content/drive/MyDrive/AWS/model_dresser.h5")
model_bed = tf.keras.models.load_model("/content/drive/MyDrive/AWS/model_bed.h5")
model_chair = tf.keras.models.load_model("/content/drive/MyDrive/AWS/model_chair.h5")

# Precising the 17 style furniture classes
class_names = ['Asian', 'Beach', 'Contemporary', 'Craftsman', 'Eclectic', 'Farmhouse', 'Industrial', 'Mediterranean', 'Midcentury', 'Modern', 'Rustic', 'Scandinavian', 'Southwestern', 'Traditional', 'Transitional', 'Tropical', 'Victorian'] 


# Create empty lists to store the results
results = []

# Loop through each file in the folder
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        # Load the image using PIL and convert it to a NumPy array
        img = Image.open(os.path.join(folder_path, filename)).resize((224, 224))
        img_array = np.array(img)
        # Preprocess the image
        img_array = tf.keras.preprocessing.image.img_to_array(img)
        img_array = tf.expand_dims(img_array, 0) # Create batch axis
        img_array = img_array / 255.0 # Normalize pixel values

        # Use each model to make a prediction on the image
        lamp_pred = model_lamp.predict(img_array)[0] if "kitchen" not in filename else None
        dresser_pred = model_dresser.predict(img_array)[0]
        bed_pred = model_bed.predict(img_array)[0] if "kitchen" not in filename else None
        chair_pred = model_chair.predict(img_array)[0] if "bedroom" not in filename else None

        # Append the results to the list
        result = [filename.split("_")[0], np.argmax(lamp_pred), np.argmax(dresser_pred), np.argmax(bed_pred), np.argmax(chair_pred)]
        results.append(result)

# Create a DataFrame from the results list
df_predict_images_test = pd.DataFrame(results, columns=["id", "lamp_prediction", "dresser_prediction", "bed_prediction", "chair_prediction"])


1/1 [==============================] - 1s 945ms/step


1/1 [==============================] - 0s 48ms/step


In [20]:
# df_predict_images.head()
# df_predict_images.sample(10)
df_predict_images_test.to_csv('dataframe_test_models_furniture_index.csv', index=True)
